In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv("Advertising.csv", index_col=0)
X, y = df.drop("Sales", axis=1), df["Sales"]
model_poly = PolynomialFeatures(3, include_bias=False)
poly_features = model_poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.33, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 19 features, indikerar på dåligt

((134, 19), (66, 19), (134,), (66,))

In [ ]:
scalar = StandardScaler()
scaled_X_train = scalar.fit_transform(X_train)
scale_X_test = scalar.transform(X_test)

print(f"Scaled X_train mean {scaled_X_train.mean():.2f} std {scaled_X_train.std():.2f}")
print(f"X_test mean {scale_X_test.mean():.2f} std {scale_X_test.std():.2f}")
# Vi försöker minska variansen

Scaled X_train mean -0.00 std 1.00
X_test mean -0.12 std 1.12


In [24]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

def ridge_regression(X, penalty=0):
    model_ridge = Ridge(alpha=penalty)
    model_ridge.fit(scaled_X_train, y_train)
    y_pred = model_ridge.predict(X)
    return y_pred

y_pred = ridge_regression(scale_X_test, penalty=0.1)

MSE = mean_squared_error(y_test, y_pred)
MAE = mean_absolute_error(y_test, y_pred)
RMSE = np.sqrt(MSE)

print(f"MSE: {MSE:.2f}")
print(f"MAE: {float(RMSE):.2f}")
print(f"MAE: {MAE:.2f}")


MSE: 0.32
MAE: 0.56
MAE: 0.43


In [ ]:
from sklearn.linear_model import Lasso

model_lasso = Lasso(alpha=0.1)
model_lasso.fit(scaled_X_train, y_train)
y_pred = model_lasso.predict(scale_X_test)

MSE = mean_squared_error(y_test, y_pred)
MAE = mean_absolute_error(y_test, y_pred)
RMSE = np.sqrt(MSE)

print(f"MSE: {MSE:.2f}")
print(f"MAE: {float(RMSE):.2f}")

model_lasso.coef_

# Man kan twaeaka för hand för att komma så långt som möjligt eller så låter vi datorn göra det.

MSE: 0.62
MAE: 0.79


array([ 1.89480144,  0.42062367,  0.        , -0.        ,  3.55216501,
        0.        ,  0.        ,  0.01110965,  0.        , -0.42677394,
       -0.        , -0.        ,  0.        ,  0.        , -0.        ,
        0.        ,  0.06706906,  0.        ,  0.        ])

In [22]:
from sklearn.linear_model import RidgeCV

model_ridgeCV = RidgeCV(alphas = [.0001, .001, .01, .1, .5, 1, 5, 10], scoring = "neg_mean_squared_error")
model_ridgeCV.fit(scaled_X_train, y_train)
print(model_ridgeCV.alpha_)

# y_pred = model_ridgeCV.predict(scale_X_test)

# MSE = mean_squared_error(y_test, y_pred)
# MAE = mean_absolute_error(y_test, y_pred)
# RMSE = np.sqrt(MSE)

0.1


In [28]:
from sklearn.linear_model import LassoCV

model_lasso_cv = LassoCV(eps = 0.001, n_alphas = 100, max_iter=10000, cv=5)
model_lasso_cv.fit(scaled_X_train, y_train)
y_pred = model_lasso_cv.predict(scale_X_test)

model_lasso_cv.alpha_, np.sqrt(mean_squared_error(y_test, y_pred)), mean_absolute_error(y_test, y_pred)

# print(model_lasso_cv.alpha_)

(0.004968802520343365, 0.5785146895301981, 0.4629188302693299)

In [ ]:
from sklearn.linear_model import ElasticNetCV

model_elasticnet_cv = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], eps = 0.001, n_alphas = 100, max_iter=10000, cv=5)
model_elasticnet_cv.fit(scaled_X_train, y_train)
y_pred = model_elasticnet_cv.predict(scale_X_test)

model_elasticnet_cv.l1_ratio_, model_elasticnet_cv.alpha_, np.sqrt(mean_squared_error(y_test, y_pred)), mean_absolute_error(y_test, y_pred)

# l1.ratio är 1 vilket betyder att lasso är den metod som är bäst för detta dataset. 

(1.0, 0.004968802520343365, 0.5785146895301981, 0.4629188302693299)